# Import packages

In [ ]:
import tqdm
from bs4 import BeautifulSoup 
import re
import urllib
import requests
import shutil 
from selenium import webdriver
import os
import time
import copy
from tqdm import tqdm
from pykospacing import spacing
import pandas as pd
import copy
import string, random ## generate random str package
from collections import OrderedDict ## repetition removal package

# Function Definition

In [ ]:
def hangul(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
    # hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+')  # 위와 동일
    result = hangul.sub('', text) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
    return str(result)

def find_categories():
    test = driver.find_elements_by_css_selector("body > div.lyr_category_lst1 > ul > li:nth-child(1) > ul > li > a")
    Categories = []
    for i in test:
        Categories.append(i.text.split("\n")[:2])
    Count_Categories = len(Categories)
    return Categories, Count_Categories

def find_post_id():
    post = driver.find_element_by_css_selector('#rego_cover > div.cover_cont > div.tit_area > div.bloger > span.thumb > a')
    r = re.compile('logNo=.*')
    string = r.findall(post.get_attribute('ng-href'))
    return re.sub("logNo=",'',"".join(string))

def random_id():
    passkey='' # an empty str key
    for x in range(10): # length of the random passkeys

        if random.choice([1,2]) == 1:
            passkey += passkey.join(random.choice(string.ascii_letters)) # upper & lower cased letter
        else:
            passkey += passkey.join(random.choice(string.digits)) # numbers
    return passkey

def Spacing_text(text_list):
    spacing_list = []
    for i in text_list:
        if len(i) < 197:
            spacing_list.append(spacing(i))
        else:
            iteration = int(len(i) / 197)
            mod = len(i) % 197
            start = 0
            end = 197
            check = 0
            while True:
                # 시행횟수 < 몫
                if check < iteration:
                    spacing_list.append(spacing(i[start:end]))
                    start+=197
                    end+=197
                    check +=1
                else:
                    # 마지막 횟수 + 나머지 더 slice 
                    spacing_list.append(spacing(i[iteration*197:(iteration*197)+mod]))
                    break
    return spacing_list

def remove_odd(x):
    x = re.sub("&nbsp"," ",x)
    x = re.sub("nbsp"," ",x)
    x = re.sub("\xa0","",x)
    x = re.sub("\u200b","",x)
    x = re.sub("\n","",x)
    x = re.sub("\t","",x)
    x = re.sub("\r","",x)
    x = re.sub('   ',' ',x)
    return x.strip()

In [ ]:
class Comment():
    def Get_Comment_Text():
        # Comment
        comment_list = []
        comments = driver.find_elements_by_class_name('u_cbox_text_wrap')
        for comment in comments:
            comment_list.append(comment.text)
        return comment_list
    
    def Get_Comment_Time():
        comment_times = driver.find_elements_by_class_name('u_cbox_date')
        comment_time = []
        for time in comment_times:
            comment_time.append(time.text)
        return comment_time
    
    def Get_Comment_ID(post_no):
        comments_href = driver.find_elements_by_class_name('u_cbox_nick')
        comment_user_list = []
        for i in range(0,len(comments_href)):
            comments_href[i].click()
            window_before = driver.window_handles[0]
            window_after = driver.window_handles[1]
            driver.switch_to_window(window_after)
            driver.implicitly_wait(10)
            comment_user_list.append(driver.current_url)
            driver.close()
            driver.switch_to_window(window_before)
        # comment단 사람 Id 추출
        comment_user_id = []

        for url in comment_user_list:

            if '?' in url: 
                comment_user_id.append(re.sub("https://blog.naver.com/","",url).split("?")[0])

            elif 'blog.me' in url:
                url_temp = re.sub("https://","",url).split(".")
                comment_user_id.append(url_temp[0])

            else:
                url_temp = re.sub("https://blog.naver.com/","",url).split("/")
                comment_user_id.append(url_temp[0])
        return comment_user_id

In [ ]:
def Post_Id_Get_1():
    foot_info = driver.find_element_by_class_name('wrap_postcomment').get_attribute('innerHTML')
    p = re.compile('id="area_sympathy.*" ')
    post_id_string = p.findall(foot_info)
    if post_id_string == []:
        p = re.compile('id="Comi.*" ')
        post_id_string = p.findall(foot_info)
    remove_quotes = re.sub('"','',str(post_id_string[0]))
    post_no = "".join(list(filter(str.isdigit,remove_quotes)))
    return post_no

In [ ]:
def Post_Id_Get_2():
    #게시글 번호 따오기

    foot_info = driver.find_element_by_id('postViewArea').get_attribute('innerHTML')
    p = re.compile('postViewArea.*"')
    post_id_string = "".join(p.findall(foot_info))

    post_id_string = re.sub('postViewArea','',post_id_string)
    post_id_string = re.sub('"','',post_id_string)
    return post_id_string

In [ ]:
def Opening_Date_Get(user_id,driver):
    opening_url = 'http://blog.naver.com/profile/intro.nhn?blogId='+ user_id
    #driver = webdriver.Chrome('.//exe_file//chromedriver.exe')
    driver.get(opening_url)
    driver.implicitly_wait(10)
    try:
        driver.find_element_by_id('category2').click()
        driver.implicitly_wait(10)
        years = driver.find_elements_by_css_selector('#post-area > div:nth-child(4) > table:nth-child(2) > tbody > tr > td > table > tbody > tr > td > table > tbody > tr:nth-child(3) > td:nth-child(2)')
        for year in years:
            if len(year.text) > 4:
                blog_opening_date = year.text
        driver.close()
        return blog_opening_date
    
    except:
        driver.close()
        return 'hide'

In [ ]:
def Comment_Sympath_Count():
    p = re.compile('[0-9]')
    sympathy_count = 'hide'
    comment_count = 'hide'
    footer_list = driver.find_elements_by_class_name('wrap_postcomment')[0].text.split('\n')
    if footer_list == ['']:
        return sympathy_count, comment_count
    else:
        for foot in footer_list:
            if '공감' in foot:
                sympathy_count = "".join(p.findall(foot))
                if sympathy_count == '':
                    sympathy_count = 0
            if '댓글' in foot:
                comment_count = "".join(p.findall(foot))
                if comment_count == '':
                    comment_count = 0
        return sympathy_count, comment_count

In [ ]:
class Comment():
    def Get_Comment_Text():
        # Comment
        comment_list = []
        comments = driver.find_elements_by_class_name('u_cbox_text_wrap')
        for comment in comments:
            comment_list.append(remove_odd(comment.text))
        return comment_list
    
    def Get_Comment_Time():
        comment_times = driver.find_elements_by_class_name('u_cbox_date')
        comment_time = []
        for time in comment_times:
            comment_time.append(time.text)
        return comment_time
    
#     def Get_Comment_ID(post_no):
#         comments_href = driver.find_elements_by_class_name('u_cbox_nick')
#         comment_user_list = []
#         for i in range(0,len(comments_href)):
#             comments_href[i].click()
#             window_before = driver.window_handles[0]
#             window_after = driver.window_handles[1]
#             driver.switch_to_window(window_after)
#             driver.implicitly_wait(10)
#             comment_user_list.append(driver.current_url)
#             driver.close()
#             driver.switch_to_window(window_before)
#         # comment단 사람 Id 추출
#         comment_user_id = []

#         for url in comment_user_list:

#             if '?' in url: 
#                 comment_user_id.append(re.sub("https://blog.naver.com/","",url).split("?")[0])

#             elif 'blog.me' in url:
#                 url_temp = re.sub("https://","",url).split(".")
#                 comment_user_id.append(url_temp[0])

#             else:
#                 url_temp = re.sub("https://blog.naver.com/","",url).split("/")
#                 comment_user_id.append(url_temp[0])
#         return comment_user_id

In [ ]:
def Sympathy_Comment_Id_Get(post_no):
    try:
        footer_list = driver.find_elements_by_class_name('wrap_postcomment')[0].text.split('\n')
        sympathy_user_id = 'hide'
        comment_user_id = 'hide'
        comment_list = 'hide'
        comment_time = 'hide'

        if footer_list ==['']:
            sympathy_user_id = 'hide'
            comment_user_id = 'hide'
            comment_list = 'hide'
            comment_time = 'hide'
            return sympathy_user_id, comment_user_id , comment_list, comment_time

        # 공감이 있는지, 댓글이 있는지 없는지 check 해주어야함.

        for number ,foot in enumerate(footer_list):

            if '공감' in foot:
                sympathy_user_id = []
                driver.find_elements_by_class_name('bu_arr')[number].click()
                # 공감한 사람 ID 찾기
                driver.implicitly_wait(3)
                sympathy_url = driver.find_element_by_id('sympathyFrm'+str(post_no)).get_attribute('src')
                driver.find_elements_by_class_name('bu_arr')[number].click()

                # 공감한 블로거 화면으로 전환
                driver.get(sympathy_url)
                driver.implicitly_wait(10)
                users_href = driver.find_elements_by_css_selector('#post-area > ul > li > div > div.bloger > span.area_text.pcol2 > strong > span > a')
                sympath_user_list = []
                for user in users_href:
                    sympath_user_list.append(user.get_attribute('href'))
                driver.back()

                # 공감한 블로거 아이디 추출
                for url in sympath_user_list:
                    if 'DomainDispatcher' in url:
                        p = re.compile('id=.*&ty')
                        string = p.findall(url)
                        string = re.sub('id=','',string[0])
                        string = re.sub('&ty','',string)
                    else:
                        url_temp = re.sub("http://blog.naver.com/","",url).split("/")
                        sympathy_user_id.append(url_temp[0])

            elif '댓글' in foot:
                comment_user_id = []
                # 댓글쓴 사람 ID들 찾기

                driver.find_elements_by_class_name('bu_arr')[number].click()
                comment_list = Comment.Get_Comment_Text()
                comment_time = Comment.Get_Comment_Time()

                comments_href = driver.find_elements_by_class_name('u_cbox_nick')
                comment_user_list = []
                for i in range(0,len(comments_href)):
                    comments_href[i].click()
                    window_before = driver.window_handles[0]
                    window_after = driver.window_handles[1]
                    driver.switch_to_window(window_after)
                    driver.implicitly_wait(10)
                    comment_user_list.append(driver.current_url)
                    driver.close()
                    driver.switch_to_window(window_before)

                # comment단 사람 Id 추출
                for url in comment_user_list:

                    if '?' in url: 
                        comment_user_id.append(re.sub("https://blog.naver.com/","",url).split("?")[0])

                    elif 'blog.me' in url:
                        url_temp = re.sub("https://","",url).split(".")
                        comment_user_id.append(url_temp[0])

                    else:
                        url_temp = re.sub("https://blog.naver.com/","",url).split("/")
                        comment_user_id.append(url_temp[-1])
            if comment_user_id == [] or comment_user_id == ['']:
                comment_list = []
                comment_time = []
            if sympathy_user_id == [] or sympathy_user_id == ['']:
                sympathy_user_id = []

        return sympathy_user_id, comment_user_id , comment_list, comment_time
    
    except:
        sympathy_user_id = []
        comment_user_id = []
        comment_list = []
        comment_time = []
        
        
        return sympathy_user_id, comment_user_id , comment_list, comment_time

In [ ]:
# user_csv 작성
csv_name = 'Naver_user_table.csv'
col_name = "\t".join(["User_id","Blog_name","Blog_nickname","Blog_info_text","Count_neighbors","Count_visitors","Categories","Count_categories","Total_post","Credibility","Source","Blog_mobile_profile_img","Blog_mobile_cover_img",'Opening_date'])

if not os.path.exists(csv_name):
    with open(csv_name, 'w',encoding='utf-8') as f:
        f.write(col_name+'\n')
        
# post_csv 작성
post_csv_name = 'Naver_post_table.csv'
post_col_name = "\t".join(["Post_id","User_id","Category","Title","Date","Structure","Structure_tag","Text","Space_text","Count_space_mistake","Map_exist","Media_count","Heart_count","Sticker_count","Comment_count"])
if not os.path.exists(post_csv_name):
    with open(post_csv_name, 'w',encoding='utf-8') as f:
        f.write(post_col_name +'\n')

# img_csv 작성
img_csv_name = 'Naver_img_table.csv'
img_col_name = "\t".join(["Img_id","Post_id"])
if not os.path.exists(img_csv_name):
    with open(img_csv_name, 'w',encoding='utf-8') as f:
        f.write(img_col_name +'\n')

# tag_csv 작성
tag_csv_name = 'Naver_tag_table.csv'
tag_col_name = "\t".join(["Post_id","Post_tag"])
if not os.path.exists(tag_csv_name):
    with open(tag_csv_name, 'w',encoding='utf-8') as f:
        f.write(tag_col_name +'\n')

# Sympathy_csv 작성
Sympathy_csv_name = 'Naver_Sympathy_table.csv'
Sympathy_col_name = "\t".join(["Post_id","Sympath_id"])
if not os.path.exists(Sympathy_csv_name):
    with open(Sympathy_csv_name, 'w') as f:
        f.write(Sympathy_col_name +'\n')
        
# Comment_csv 작성
Comment_csv_name = 'Naver_Comment_table.csv'
Comment_col_name = "\t".join(["Post_id","Comment_id","Comment_Text","Comment_Time"])
if not os.path.exists(Comment_csv_name):
    with open(Comment_csv_name, 'w',encoding='utf-8') as f:
        f.write(Comment_col_name +'\n')

In [ ]:
Base_path = os.getcwd()

# 기본 path설정
Naver_folder = "C:\\Users\\YongGeol\\Desktop\Data\\Naver\\" 
Base_path = 'C:\\workspace'
Chrome_path = 'C:\\workspace\\chromedriver.exe'

## Naver Keyword

In [ ]:
Naver_Post_set = pd.read_csv('Total_Naver_Post_id.csv',sep='\t',encoding='utf-8',names=['User_id','Category','Post_id'])

In [ ]:
Naver_Post_set = Naver_Post_set[10539:]

In [ ]:
Naver_Post_set['User_id'].iloc[1]

In [ ]:
## Path 설정
move_path =  Naver_folder

## User_Table Crwaling
#------------------------------------------------------------------------------------------------------------------------------------------------------------
# user_id, post_id zip, remove_duplicated


# source credibility
Credibility = 1
Source = "Naver"

# User_table
for number in tqdm(range(0,len(Naver_Post_set))):

    User_id = Naver_Post_set['User_id'].iloc[number]
    Post_id = Naver_Post_set['Post_id'].iloc[number]
    Category =Naver_Post_set['Category'].iloc[number]
    Source = Source
    Credibility = Credibility
    existed_user_id = os.listdir(Naver_folder)
    
    try:
        if User_id not in existed_user_id:
            url = "http://blog.naver.com/PostView.nhn?blogId="+User_id+ "&logNo=" + Post_id +"&redirect=Dlog&widgetTypeCall=true"
            mobile_url = "http://m.blog.naver.com/PostView.nhn?blogId="+ User_id

            # get mobile_url
            driver = webdriver.Chrome(Chrome_path)
            driver.get(mobile_url)
            driver.implicitly_wait(3)

            # 존재하지 않는 게시물 클릭
            driver.find_element_by_class_name("btn_area").click()
            driver.implicitly_wait(10)
            time.sleep(1)
            # Blog_name, Blog_nickname, Blog_mobile_profile_img, Blog_info_text
            Blog_name = driver.find_element_by_css_selector('#rego_cover > div.cover_cont > div.tit_area > h2 > a > span').text
            Blog_nickname =driver.find_element_by_class_name("user_name").text

            try:
                post = driver.find_element_by_css_selector('#rego_cover > div.cover_cont > div.tit_area > div.bloger > span.thumb > a')
                Blog_mobile_profile_img_url = post.get_attribute('ng-href')  
            except:
                post = driver.find_element_by_css_selector('#rego_cover > div.cover_cont > div.tit_area > div.bloger > span.thumb > a')
                Blog_mobile_profile_img_url = post.get_attribute('href')

            try:
                Blog_info_text = driver.find_element_by_class_name("text").text

            except:
                Blog_info_text = ""
            Blog_info_text = remove_odd(Blog_info_text)    

            # Count_neighbors
            neighbors_string = re.sub(",","",driver.find_element_by_class_name("count_buddy").text)
            Count_neighbors = int(re.findall('\d+', neighbors_string)[0])

            # Count_visitors
            visitor_stirng = driver.find_elements_by_class_name('count')[0].text
            Count_visitors = re.sub(",","",visitor_stirng.split("전체")[1]).strip()

            # download_img
            Mobile_cover_img_url = driver.find_element_by_class_name("cover_img").get_attribute('bg-lazy-img')

            # category 버튼 클릭
            driver.find_element_by_css_selector("#rego_cover > div.cover_cont > div.btn_area > div > div:nth-child(2) > a > span.txt").click()
            driver.implicitly_wait(3)

            # Total_Post, Categories, Count_Categories, 

            Categories, Count_categories =  find_categories()
            Total_post = driver.find_element_by_class_name("num").text
            #driver.close()
            
            try:
                Opening_date = Opening_Date_Get(User_id,driver)
            except:
                print("error")
                Opening_date = 'hide'

            os.chdir(move_path)

            # make category_folder
            folder_name = User_id
            if not os.path.exists(folder_name):
                os.makedirs(folder_name)

            User_path = os.path.join(move_path,User_id)
            os.chdir(User_path)

            Save_path = Naver_folder + '\\' + folder_name
            os.chdir(Save_path)
            cover_img = "Cover_" + User_id + ".jpg"
            Blog_mobile_profile_img = 'Naver' +'\\'+ User_id + "\\" + cover_img

            profile_img = "Profile_" + User_id + ".jpg"
            Blog_mobile_cover_img =  'Naver' +'\\' + User_id +  "\\" + profile_img

            #download_img
            profile_cover_image_check = os.listdir(os.path.join(move_path,User_id))
            if "Profile_" or "Cover_" not in profile_cover_image_check: 
                urllib.request.urlretrieve(Mobile_cover_img_url,cover_img)
                urllib.request.urlretrieve(Blog_mobile_profile_img_url,profile_img)

            os.chdir(Base_path)

            save_content_list = [User_id,Blog_name,Blog_nickname,Blog_info_text,Count_neighbors,Count_visitors,Categories,Count_categories,Total_post,Credibility,Source,Blog_mobile_profile_img,Blog_mobile_cover_img,Opening_date]
            save_content = "\t".join(list(map(str,save_content_list)))

            with open(csv_name, 'a', encoding='utf-8') as f:
                f.write(save_content+"\n")
                
    except:
        print('user_error')
        pass

# Post & Tag & Img Table_Crwaling        
#------------------------------------------------------------------------------------------------------------------------------------------------------------
    existed_post_id = os.listdir(Naver_folder+'\\'+ User_id)
    
    if Post_id not in existed_post_id:

        try:
            url = "http://blog.naver.com/PostView.nhn?blogId="+User_id+ "&logNo=" + Post_id +"&redirect=Dlog&widgetTypeCall=true"
            r = requests.get(url)
            bs = BeautifulSoup(re.sub('&nbsp;',' ',r.text).encode("utf-8"), "html.parser")

            #title
            Title = bs.find("h3", {"class": "se_textarea"})
            if (Title == None):
                Title = bs.find("span", {"class": "pcol1 itemSubjectBoldfont"})
            if (Title != None):
                Title = Title.text.strip()
            else:
                Title = "TITLE ERROR"

            #date
            # Append_value
            Date = bs.find("span", {"class": "se_publishDate pcol2 fil5"})

            if Date == None:
                Date = bs.find("p",{"class":"date fil5 pcol2 _postAddDate"})
            if Date == None:
                Date = bs.find("span",{"class":"se_publishDate pcol2"})

            Date_text = re.sub("\n","",Date.text)
            Date = re.sub("\t","",Date_text)

            #Tag, Content_structure, Text

            # structure
            structure = bs.find("div", {"id": "postViewArea"})
            if structure == None:
                structure = bs.find("div",{"class","se_component_wrap sect_dsc __se_component_area"})
            structure_tag = structure.find_all(['p','img'])


            # only tag & texf extract
            tag_list = []
            structure_list = []
            text_list = []

            for i in structure_tag:
                # p_tag만 불러오기
                if "<p" in (str(i)):
                    tag_list.append('<p>')
                    structure_list.append('<p>')
                    # img만 있을 때

                    if '<img' in str(i):
                        for j in i:
                            try:
                                if len(j.text)>1:
                                    tag_list.append('<br>')
                                    structure_list.append('<br>')
                                    structure_list.append(j.text)
                                    text_list.append(j.text)
                            except:
                                pass

                    # img가 아닌 경우 span tag가 더 있을 때
                    elif '<span' in str(i):
                        for j in i:
                            if '<br' in str(j):
                                structure_list.append(j.text)
                                text_list.append(j.text)
                                # br_tag가 2개 이상 있을 때

                                if len(j.findAll('br'))>2:
                                    for _ in range(0,len(j.findAll('br'))):
                                        tag_list.append('<br>')
                                        structure_list.append('<br>')

                                # br_tag가 1개 있을 때
                                else:
                                    tag_list.append('<br>')
                                    structure_list.append('<br>')

                            # span은 있지만 br tag가 없을 때       
                            else:
                                try:
                                    structure_list.append(j.text)
                                    text_list.append(j.text)
                                except:
                                    pass

                    # 그냥 p_tag만 있을 때 br_tag 추가
                    else:
                        # 글이 있을 때
                        if len(i.text) > 1:
                            structure_list.append(i.text)
                            text_list.append(i.text)

                        # 글 없이 br tag만 있을 때
                        else:
                            tag_list.append('<br>')
                            structure_list.append(i.text)
                            structure_list.append('<br>')
                            text_list.append(i.text)

                    # P_tag 끝맽음      
                    tag_list.append('</p>')
                    structure_list.append('</p>')

                else:
                    tag_list.append('<img>')
                    structure_list.append('<img>')


            structure_list = list(map(remove_odd,structure_list))
            text_list = list(map(remove_odd,text_list))
            filter_text = list(filter(lambda x: len(x)>1 ,text_list))

            Structure = "|".join(list(filter(lambda x: len(x)>1 ,structure_list)))
            Structure = remove_odd(Structure)

            Text = " ".join(list(filter(lambda x: len(x)>1 ,map(lambda x : x.strip(),text_list))))
            Text = remove_odd(Text)


            Space_text = " ".join(Spacing_text(filter_text))
            Space_text = remove_odd(Space_text)

            Count_space_mistake = len(Space_text)-len(Text)


            # only tag
            Structure_tag = "|".join(tag_list)
            Structure_tag = remove_odd(Structure_tag)


            # image download    
            imgs = structure.find_all('img')

            User_path = os.path.join(move_path,User_id)
            os.chdir(User_path)

            folder_name = Post_id
            if not os.path.exists(folder_name):
                os.makedirs(folder_name)


            Save_path = os.path.join(User_path,Post_id)


            os.chdir(Base_path)

            Map_exist = 0
            Sticker_count = 0


            # Sticker_count로 수정
            sticker_img = structure.find_all('a')
            for i in sticker_img:
                if 'sticker' in str(i):
                    Sticker_count += 1

            # image download    
            imgs = structure.find_all('img')

            for img in imgs:


                img_name =  User_id + "_" + random_id() + ".jpg"

                try:
                    img_url = re.sub("\u200b","",str(img['src']))
                    try:
                        urllib.request.urlretrieve(img_url,img_name)
                        shutil.move(img_name,Save_path)

                    except UnicodeEncodeError:
                        try:
                            if 'map' not in str(img_url):
                                driver = webdriver.Chrome(Chrome_path)
                                driver.get(url)
                                driver.implicitly_wait(3)
                                img = driver.find_element_by_tag_name('img')
                                src = re.sub("\u200b","",str(img.get_attribute('src')))
                                urllib.request.urlretrieve(src, Save_path)
                                shutil.move(img_name,move_folder)
                                driver.close()
                            else:
                                Map_exist = 1
                        except:
                            pass
                    except:
                        pass



                    Img_id = "Naver" + '\\' + User_id + '\\'+ Post_id +'\\' + img_name
                    save_img_content = Img_id + "\t" + Post_id

                    with open(img_csv_name, 'a', encoding='utf-8') as f:
                        f.write(save_img_content + "\n")
                except:
                    pass

            # Post_tag, Media_count
            driver = webdriver.Chrome(Chrome_path)
            driver.get(url)
            driver.implicitly_wait(20)

            # Comment, Sympath_table
            Sympathy_user_id, Comment_user_id , Comment_list, Comment_time = Sympathy_Comment_Id_Get(Post_id)

            if Sympathy_user_id != 'hide' or Sympathy_user_id != []:
                for Sympath_id in Sympathy_user_id:
                    if Sympath_id == 'h':
                        break
                    with open(Sympathy_csv_name, 'a', encoding='utf-8') as f:
                        f.write(Post_id+'\t'+Sympath_id + "\n")

            if Comment_user_id != 'hide' or Comment_user_id != []:
                for i in range(0,len(Comment_user_id)):
                    with open(Comment_csv_name, 'a', encoding='utf-8') as f:
                        f.write(Post_id+'\t' + Comment_user_id[i] +'\t'+ Comment_list[i] +'\t' + Comment_time[i] + "\n") 



            #extract comment_count
            try:
                Heart_count, Comment_count = Comment_Sympath_Count()
            except:
                Heart_count = 'hide'
                Comment_count = 'hide'

            Media_count = 0
            Media_check = driver.find_elements_by_tag_name('iframe')

            for Media in Media_check:
                if 'Player' in str(Media.get_attribute('src')):
                    Media_count += 1
            try:
                tag_list = re.sub('\n','',driver.find_element_by_class_name('wrap_tag').text).strip().split('#')[1:]
            except:
                tag_list = []

            for Post_tag in tag_list:
                save_tag = Post_id + "\t" + Post_tag
                with open(tag_csv_name, 'a', encoding='utf-8') as f:
                    f.write(save_tag + "\n")


            # make a save list
            save_post = "\t".join(list(map(str,[Post_id,User_id,Category,Title,Date,Structure,Structure_tag,Text,Space_text,Count_space_mistake,Map_exist,Media_count,Heart_count,Sticker_count,Comment_count])))

            with open(post_csv_name, 'a', encoding='utf-8') as f:
                    f.write(save_post + "\n")

            driver.close()
        except:
            print('post_error')
            continue